In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [14]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})
df = df.iloc[:1171]
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')
df.set_index('Date', inplace=True)
df =df.apply(pd.to_numeric, errors='coerce')
df

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84
1926-11-01,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11
...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-4.57,-2.58,-7.08,3.17,-5.98,-3.22,-5.68,-4.71,-5.04,-3.41
2023-10-01,-3.53,-17.88,-2.76,-6.24,-1.78,-0.18,0.47,-4.58,1.12,-2.53
2023-11-01,5.02,15.76,8.68,-1.29,11.96,6.97,7.18,5.87,5.08,10.41


In [4]:
def rolling_pf_SR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()



In [18]:
def rolling_pf_SR_NS(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [6]:
def rolling_pf_VAR(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        var = window.var()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / var) / np.sum(1 / var)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,0.206083,0.04888,0.072338,0.081887,0.045994,0.144641,0.085687,0.135005,0.06028,0.119205
1931-08-01,0.205698,0.05135,0.071337,0.082631,0.045507,0.143679,0.086623,0.136505,0.060732,0.115938
1931-09-01,0.207208,0.051302,0.071397,0.080634,0.045456,0.143691,0.086422,0.137075,0.060698,0.116115
1931-10-01,0.18676,0.055274,0.076448,0.085237,0.049938,0.15044,0.088519,0.12458,0.063644,0.11916
1931-11-01,0.186368,0.05559,0.077585,0.082843,0.050417,0.152696,0.086379,0.122496,0.064102,0.121524
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.168357,0.01843,0.093046,0.028287,0.08318,0.106461,0.101547,0.159128,0.1545,0.087063
2023-09-01,0.166503,0.018403,0.092855,0.028406,0.084031,0.107066,0.104113,0.161061,0.150882,0.086681
2023-10-01,0.1659,0.01875,0.091968,0.028902,0.083724,0.108454,0.103704,0.160613,0.150251,0.087734


In [7]:
def rolling_pf_VOL(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        std = window.std()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / std) / np.sum(1 / std)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [8]:
def rolling_pf_inverse_volatility(returns, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        sigma = window.cov()
        volatilities = np.sqrt(np.diag(sigma))  # Calculate volatilities
        
        def inverse_volatility(w):
            w = np.array(w)
            inv_vol = 1 / volatilities
            weighted_inv_vol = np.dot(w, inv_vol)
            return -weighted_inv_vol  # Maximize instead of minimize

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(inverse_volatility, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [9]:
#def rolling_pf_SameWeights(returns, window_size):

In [10]:
#def rolling_pf_MarketCap(returns, window_size):

In [15]:
def rolling_pf_minVAR(returns, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def min_volatility(w):
            w = np.array(w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            return V

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(min_volatility, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

rolling_pf_minVAR(df, window_size=60)


,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,1.332805,-0.019403,-0.58611,0.16511,-0.138511,0.532527,-0.004057,0.010454,-0.143604,-0.149211
1931-08-01,1.340013,-0.052891,-0.585125,0.168483,-0.104382,0.54596,0.001161,0.013572,-0.17216,-0.154631
1931-09-01,1.354091,-0.049739,-0.584361,0.160314,-0.099685,0.52808,-0.020586,0.023067,-0.186763,-0.124418
1931-10-01,1.463834,-0.051756,-0.658393,0.134347,-0.071741,0.698924,0.008237,-0.138402,-0.301244,-0.083805
1931-11-01,1.453658,-0.066327,-0.645824,0.114033,-0.07706,0.710323,0.00282,-0.14712,-0.310937,-0.033566
...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.616927,-0.039147,-0.554652,-0.097262,-0.034668,0.150064,0.176142,0.446734,0.256766,0.079096
2023-10-01,0.619842,-0.037744,-0.610515,-0.091003,-0.046492,0.160505,0.183755,0.452087,0.260267,0.109298
2023-11-01,0.629606,-0.03973,-0.637665,-0.09441,-0.051275,0.177866,0.196289,0.431293,0.269916,0.11811
